In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from CKKS_Scheme.ckks_encoder import CKKSEncoder
from CKKS_Scheme.ckks import Params,CKKS
from utils.plaintext import Plaintext
from utils.poly import Polynomial
import math 
from math import floor

In [2]:

params = Params(8,  1<<600,1<<1200)
ckks= CKKS(params)
public_key,private_key=ckks.generate_keys()


In [3]:
def read_dataset(url,col):
  df=pd.read_csv(url)
#   df.drop("Price", axis=1, inplace=True)
  X = df.iloc[:, :col]
  col_name=df.columns[col]
  y = df[col_name]
  X_train, X_test, y_train, y_test = train_test_split(
  X,y , random_state=104,test_size=0.25, shuffle=True)
  X_train=np.asarray(X_train).astype('float32')
  X_test=np.asarray(X_test).astype('float32')
  y_train=np.asarray(y_train).astype('float32')
  y_test=np.asarray(y_test).astype('float32')
  scaler = StandardScaler()
  X_train = scaler.fit_transform(X_train)
  X_test = scaler.transform(X_test)
  y_train=y_train.reshape(-1,1)
  y_test=y_test.reshape(-1,1)
  y_train = scaler.fit_transform(y_train)
  y_test = scaler.transform(y_test)
  y_train=y_train[:,0]
  y_test=y_test[:,0]
  print(X_test.shape)
  return X_train, X_test, y_train, y_test

In [4]:
X_train, X_test, y_train, y_test = read_dataset('../dataset/USA_Housing.csv',5)

(1250, 5)


In [5]:
y_train

array([-2.349955  , -0.42323497,  2.6654403 , ..., -0.6044362 ,
        1.3268416 ,  0.35598105], dtype=float32)

In [6]:
def custom_relu(x):
    return x*x
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(5,)),

    tf.keras.layers.Dense(5),
    tf.keras.layers.Activation(custom_relu),
    tf.keras.layers.Dense(1),
])
model.compile(optimizer='adam', loss='mse')
model.fit(X_train, y_train, epochs=30, batch_size=16)
loss = model.evaluate(X_test, y_test, verbose=0)
print(f"Test Loss: {loss:.2f}")

Epoch 1/30
235/235 [==============================] - 1s 2ms/step - loss: 5.5665
Epoch 2/30
235/235 [==============================] - 0s 2ms/step - loss: 2.1041
Epoch 3/30
235/235 [==============================] - 0s 2ms/step - loss: 1.2636
Epoch 4/30
235/235 [==============================] - 0s 2ms/step - loss: 0.8903
Epoch 5/30
235/235 [==============================] - 0s 2ms/step - loss: 0.6433
Epoch 6/30
235/235 [==============================] - 0s 2ms/step - loss: 0.4476
Epoch 7/30
235/235 [==============================] - 0s 2ms/step - loss: 0.3048
Epoch 8/30
235/235 [==============================] - 0s 2ms/step - loss: 0.2149
Epoch 9/30
235/235 [==============================] - 0s 2ms/step - loss: 0.1632
Epoch 10/30
235/235 [==============================] - 0s 2ms/step - loss: 0.1333
Epoch 11/30
235/235 [==============================] - 0s 2ms/step - loss: 0.1150
Epoch 12/30
235/235 [==============================] - 0s 2ms/step - loss: 0.1040
Epoch 13/30
235/235 [====

In [7]:
def create_model_weights_file(model):
  i=0
  for layer in model.layers:
    if('activation' in layer.name):
      continue
    w = (layer.get_weights()[0]).T
    
    b = (layer.get_weights()[1]).T
    file_name = 'layer'+str(i)+'.npy'
    with open(file_name, 'wb') as f:
        np.save(f, w)
        np.save(f, b)
    i=i+1


In [8]:
create_model_weights_file(model)

In [9]:
def encode_value(a,scale):
    encoder=CKKSEncoder(16,scale)
    a_encoded=encoder.encode(np.array([a]*4))
    a_encoded=Polynomial(a_encoded.coef)
    a_encoded.poly_floor()
    a_pt=Plaintext(a_encoded,scale)
    return a_pt
def encrypt_input(x,scale):
    res=[]
    for item in x:
        item_pt=encode_value(item,scale)
        item_encrypted=public_key.encrypt(item_pt)
        res.append(item_encrypted)
    return res
    
    

In [10]:
class Model:
    def __init__(self,n_features,n_layers,n_layer_neurons,layer_activations,layer_weight_files,encrypted):
        self.n_features=n_features
        self.n_layers=n_layers
        self.n_layer_neurons=n_layer_neurons
        self.layer_activations=layer_activations
        self.weights=[]
        self.biases=[]
        self.encrypted=encrypted
        layer_number=0
        b_scale=1<<20
        for wt_file in layer_weight_files:
            with open(wt_file, 'rb') as f:
              if self.encrypted==True:
                  w=np.load(f)
                  r,c=np.shape(w)
                  res_w=[]
                  for i in range(r):
                      row=[]
                      for j in range(c):
                          row.append(encode_value(w[i][j],b_scale))
                      res_w.append(row)
                  b=np.load(f)
                  res_b=[]
                #   b_scale=b_scale*(1<<12)
                  for i in b:
                        res_b.append(encode_value(i,b_scale))
                       
                #   if self.layer_activations[layer_number]=='relu':
                #         b_scale=b_scale**2
                  self.weights.append(res_w)
                  self.biases.append(res_b)
                  layer_number+=1
                   
              
              else:
                self.weights.append(np.load(f).tolist())
                self.biases.append(np.load(f).tolist())
                
    def __matmul(self,a,b):
        ## Input: a to be 2-D (n_layer_neurons[layer_no],n_features)and b to be 1-D list
        # Output : a 1-D list 
        output=[]
        for i in range(len(a)):
            res=a[i][0]*b[0]
            for j in range(1,len(b)):
                res=a[i][j]*b[j]+res
            output.append(res)
        return output
    
    def __matadd(self,a,b):
        ## Input: a and b to be 1-D list
        # Output : a 1-D list 
        ## For memory saving b is modified in calculations so be careful about order of inputs
        
        for i in range(len(a)):
            b[i]=b[i]+a[i]
        return b  
    
    def infer(self,x):
        for i in range(self.n_layers):
            
            new_x=self.__matmul(self.weights[i],x)
            new_x=self.__matadd(self.biases[i],new_x)
            
            if(self.layer_activations[i]=='relu'):
               
                x=[t**2 for t in new_x]
               
            else:  
                x=new_x  
        return x[0]


In [11]:
model_encrypted=Model(30,2,[2,1],['relu','none'],['layer0.npy','layer1.npy'],True)
model_unencrypted=Model(30,2,[2,1],['relu','none'],['layer0.npy','layer1.npy'],False)

In [12]:
normal_mse=0
encrypted_mse=0
limit=len(X_test)
for i in range(limit):
    x=X_test[i].tolist()
    x_encrypted=encrypt_input(x,1<<12)
    normal_res=model_unencrypted.infer(x)
    pt_dash=private_key.decrypt(model_encrypted.infer(x_encrypted))
    pt=np.polynomial.Polynomial(pt_dash.poly.convert_to_list())
    encoder=CKKSEncoder(16,pt_dash.scale)
    pt=encoder.decode(pt)
    encrypted_res=pt[0].real
    #print('Unencryted Inference:'+str(normal_res)+" Encrypted Inference:"+str(encrypted_res)+" Actual Answer:"+str(y_test[i]))
    normal_mse+=(y_test[i]-normal_res)**2
    encrypted_mse+=(y_test[i]-encrypted_res)**2

print('Loss on unencrypted Test Data:'+str(normal_mse/limit))
print('Loss on encrypted test data:'+str(encrypted_mse/limit))

Loss on unencrypted Test Data:0.08112754622877502
Loss on encrypted test data:0.08115129906685117
